In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('MMSA_2021.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227792 entries, 0 to 227791
Columns: 152 entries, DISPCODE to MMSANAME
dtypes: float64(151), object(1)
memory usage: 264.2+ MB


In [4]:
data.head()

,DISPCODE,STATERE1,CELPHON1,LADULT1,COLGSEX,LANDSEX,RESPSLCT,SAFETIME,CADULT1,CELLSEX,...,_FRTLT1A,_VEGLT1A,_FRT16A,_VEG23A,_FRUITE1,_VEGETE1,_MMSA,_MMSAWT,SEQNO,MMSANAME
0,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,2.0,1.0,1.0,1.0,5.397605e-79,5.397605e-79,10100.0,90.030265,2.021000e+09,"b'Aberdeen, SD, Micropolitan Statistical Area'"
1,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,2.0,1.0,1.0,1.0,5.397605e-79,5.397605e-79,10100.0,29.643176,2.021000e+09,"b'Aberdeen, SD, Micropolitan Statistical Area'"
2,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,...,1.0,1.0,1.0,1.0,5.397605e-79,5.397605e-79,10100.0,37.000680,2.021000e+09,"b'Aberdeen, SD, Micropolitan Statistical Area'"
3,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,5.397605e-79,5.397605e-79,10100.0,49.702755,2.021000e+09,"b'Aberdeen, SD, Micropolitan Statistical Area'"
4,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,...,1.0,1.0,1.0,1.0,5.397605e-79,5.397605e-79,10100.0,15.396983,2.021000e+09,"b'Aberdeen, SD, Micropolitan Statistical Area'"


In [5]:
isna_data = data.isna().sum()
for column, count in isna_data.items():
    print(f"{column}: {count}")

DISPCODE: 0
STATERE1: 172268
CELPHON1: 172268
LADULT1: 172268
COLGSEX: 227774
LANDSEX: 204447
RESPSLCT: 196968
SAFETIME: 55524
CADULT1: 55524
CELLSEX: 55525
HHADULT: 55525
SEXVAR: 0
GENHLTH: 2
PHYSHLTH: 2
MENTHLTH: 0
POORHLTH: 104941
PRIMINSR: 1
PERSDOC3: 0
MEDCOST1: 1
CHECKUP1: 0
EXERANY2: 0
BPHIGH6: 0
BPMEDS: 143010
CHOLCHK3: 0
TOLDHI3: 29849
CHOLMED3: 30130
CVDINFR4: 0
CVDCRHD4: 0
CVDSTRK3: 0
ASTHMA3: 0
ASTHNOW: 195581
CHCSCNCR: 0
CHCOCNCR: 0
CHCCOPD3: 0
ADDEPEV3: 0
CHCKDNY2: 0
DIABETE4: 0
DIABAGE3: 199821
HAVARTH5: 0
ARTHEXER: 158813
ARTHEDU: 158813
LMTJOIN3: 158813
ARTHDIS2: 158813
JOINPAI2: 158813
MARITAL: 1
EDUCA: 0
RENTHOM1: 1
NUMHHOL3: 172269
NUMPHON3: 211707
CPDEMO1B: 170
VETERAN3: 949
EMPLOY1: 2047
CHILDREN: 3540
INCOME3: 5154
PREGNANT: 182883
WEIGHT2: 6747
HEIGHT3: 7408
DEAF: 8316
BLIND: 8973
DECIDE: 9547
DIFFWALK: 10201
DIFFDRES: 10646
DIFFALON: 11222
SMOKE100: 12001
SMOKDAY2: 145458
USENOW3: 12408
ECIGNOW1: 13013
ALCDAY5: 13868
AVEDRNK3: 113057
DRNK3GE5: 113425
MAXDRNKS: 

In [6]:
data.dropna().shape

(0, 152)

In [12]:
columns = [
    '_RFHLTH',
    '_PHYS14D',
    '_MENT14D',
    '_HLTHPLN',
    '_HCVU652',
    '_TOTINDA',
    '_RFHYPE6',
    '_CHOLCH3',
    '_RFCHOL3',
    '_MICHD',
    '_DRDXAR3',
    '_LMTACT3',
    '_LMTWRK3',
    '_SMOKER3',
    'DRNKANY5',
    '_RFBING5',
    '_DRNKWK1',
    '_RFDRHV7',
    '_FLSHOT7',
    '_PNEUMO3',
    '_AIDTST4',
    'FTJUDA2_',
    'FRUTDA2_',
    'GRENDA1_',
    'FRNCHDA_',
    'POTADA1_',
    'VEGEDA2_',
    '_FRUTSU1',
    '_VEGESU1',
    '_FRTLT1A',
    '_VEGLT1A',
    '_INCOMG1',
    '_EDUCAG',
    '_RFBMI5',
    '_BMI5CAT',
    '_BMI5',
    'WTKG3',
    '_AGE_G',
    '_AGE65YR',
    '_SEX',
    '_RACEPRV',
    '_RACEGR3',
    '_RACEG21',
    '_RACE',
    '_HISPANC',
    '_MRACE1',
    '_PRACE1',
]

In [13]:
for col in columns:
    if col in data.columns:
        print(col)
    else:
        print(f'{col} does not exist')

_RFHLTH
_PHYS14D
_MENT14D
_HLTHPLN
_HCVU652
_TOTINDA
_RFHYPE6
_CHOLCH3
_RFCHOL3
_MICHD
_DRDXAR3
_LMTACT3
_LMTWRK3
_SMOKER3
DRNKANY5
_RFBING5
_DRNKWK1
_RFDRHV7
_FLSHOT7
_PNEUMO3
_AIDTST4
FTJUDA2_
FRUTDA2_
GRENDA1_
FRNCHDA_
POTADA1_
VEGEDA2_
_FRUTSU1
_VEGESU1
_FRTLT1A
_VEGLT1A
_INCOMG1
_EDUCAG
_RFBMI5
_BMI5CAT
_BMI5
WTKG3
_AGE_G
_AGE65YR
_SEX
_RACEPRV
_RACEGR3
_RACEG21
_RACE
_HISPANC
_MRACE1
_PRACE1


In [14]:
data_copy = data[columns].copy()

In [15]:
isna_data = data_copy.isna().sum()
for column, count in isna_data.items():
    print(f"{column}: {count}")

_RFHLTH: 0
_PHYS14D: 0
_MENT14D: 0
_HLTHPLN: 0
_HCVU652: 0
_TOTINDA: 0
_RFHYPE6: 0
_CHOLCH3: 0
_RFCHOL3: 29981
_MICHD: 2308
_DRDXAR3: 1480
_LMTACT3: 1480
_LMTWRK3: 1480
_SMOKER3: 0
DRNKANY5: 0
_RFBING5: 0
_DRNKWK1: 0
_RFDRHV7: 0
_FLSHOT7: 149524
_PNEUMO3: 149524
_AIDTST4: 17092
FTJUDA2_: 24654
FRUTDA2_: 24097
GRENDA1_: 24449
FRNCHDA_: 24886
POTADA1_: 26756
VEGEDA2_: 26817
_FRUTSU1: 27859
_VEGESU1: 32613
_FRTLT1A: 0
_VEGLT1A: 0
_INCOMG1: 0
_EDUCAG: 0
_RFBMI5: 0
_BMI5CAT: 26341
_BMI5: 26341
WTKG3: 22711
_AGE_G: 0
_AGE65YR: 0
_SEX: 0
_RACEPRV: 0
_RACEGR3: 0
_RACEG21: 0
_RACE: 0
_HISPANC: 0
_MRACE1: 0
_PRACE1: 0
